# 1

In [1]:
import sys
!{sys.executable} -m pip install graphviz

In [2]:
!{sys.executable} -m pip install pydot

In [3]:
from sklearn import tree
from sklearn.datasets import load_iris
import pydot
import numpy as np
import random
from numpy.random import RandomState
from scipy import stats
import math
import graphviz
import pandas as pd

df_data = pd.read_csv('skysurvey/training_data.csv')
df_class = pd.read_csv('skysurvey/training_class.csv')

list1=['stars','galaxies','quasars']
target_names = np.array(list1)
list2=['ra', 'dec', 'u', 'g', 'r', 'i', 'z', 'run', 'rerun', 'camcol', 'fifield', 'specobjid', 'redshift', 'plate', 'mjd','fifiberid']
feature_names = np.array(list2)

clf = tree.DecisionTreeClassifier(criterion='gini',min_samples_leaf=0.01,random_state=RandomState(2018))
clf = clf.fit(df_data, df_class)


#to visualize the decision tree
dot_data = tree.export_graphviz(clf, out_file=None, 
                         feature_names=feature_names,  
                         class_names=target_names,  
                         filled=True, rounded=True,  
                         special_characters=True)  

graphv = graphviz.Source(dot_data)  

#print the decision tree in a pdf file
#from sklearn.externals.six import StringIO
from six import StringIO
import pydot
import pydotplus
import graphviz

import pydot
dot_data = StringIO()
tree.export_graphviz(clf, out_file=dot_data, feature_names=feature_names,  
                         class_names=target_names,  
                         filled=True, rounded=True,  
                         special_characters=True  
                    )
graph = pydot.graph_from_dot_data(dot_data.getvalue())
graph[0].write_pdf("SkySurvey.pdf")

# 2

In [4]:
print ("training errors:"+str((1-clf.score(df_data,df_class))))

training errors:0.01130113011301126


In [5]:
leaf_count=0
for i in range (clf.tree_.node_count):
    if clf.tree_.children_left[i]==-1:
        leaf_count+=1
print("Number of leaf nodes in the tree:" + str(leaf_count))

Number of leaf nodes in the tree:28


In [6]:
print("generalization error:"+str((1-clf.score(df_data,df_class))*len(df_data)+0.5*(leaf_count)))

generalization error:126.99999999999959


# 3

In [7]:
clf.get_depth()

11

In [8]:
for i in range(1,12):
    clf = tree.DecisionTreeClassifier(criterion='gini',min_samples_leaf=0.01,random_state=RandomState(2018),max_depth=i)
    clf = clf.fit(df_data, df_class)
    leaf_count=0
    for i in range (clf.tree_.node_count):
        if clf.tree_.children_left[i]==-1:
            leaf_count+=1
    print("generalization error when max_depth="+str(i)+': '+str((1-clf.score(df_data,df_class))*len(df_data)+0.5*(leaf_count)))

generalization error when max_depth=2: 878.0000000000002
generalization error when max_depth=6: 114.99999999999959
generalization error when max_depth=14: 116.99999999999959
generalization error when max_depth=24: 119.49999999999959
generalization error when max_depth=32: 121.49999999999959
generalization error when max_depth=34: 121.99999999999959
generalization error when max_depth=38: 122.99999999999959
generalization error when max_depth=44: 124.49999999999959
generalization error when max_depth=50: 125.99999999999959
generalization error when max_depth=52: 126.49999999999959
generalization error when max_depth=54: 126.99999999999959


In [10]:
clf = tree.DecisionTreeClassifier(criterion='gini',min_samples_leaf=0.01,random_state=RandomState(2018),max_depth=2)
clf = clf.fit(df_data, df_class)
dot_data = StringIO()
tree.export_graphviz(clf, out_file=dot_data, feature_names=feature_names,  
                         class_names=target_names,  
                         filled=True, rounded=True,  
                         special_characters=True  
                    )
graph = pydot.graph_from_dot_data(dot_data.getvalue())
graph[0].write_pdf("max_length=2.pdf")

# 7

In [11]:
n_nodes = clf.tree_.node_count
children_left = clf.tree_.children_left
children_right = clf.tree_.children_right
feature = clf.tree_.feature
threshold = clf.tree_.threshold
impurity = clf.tree_.impurity
value = clf.tree_.value
samples = clf.tree_. n_node_samples
print(feature)
print(children_left)
print(children_right)
print(impurity)
print(samples)
print(value)

[12 12 -2 -2 12 -2 -2]
[ 1  2 -1 -1  5 -1 -1]
[ 4  3 -1 -1  6 -1 -1]
[0.57058107 0.01144446 0.02244609 0.00179011 0.25008085 0.02522176
 0.05953854]
[9999 4171 1938 2233 5828 5014  814]
[[[4.151e+03 4.998e+03 8.500e+02]]

 [[4.147e+03 2.300e+01 1.000e+00]]

 [[1.916e+03 2.200e+01 0.000e+00]]

 [[2.231e+03 1.000e+00 1.000e+00]]

 [[4.000e+00 4.975e+03 8.490e+02]]

 [[4.000e+00 4.950e+03 6.000e+01]]

 [[0.000e+00 2.500e+01 7.890e+02]]]


In [12]:
import sys
!{sys.executable} -m pip install anytree
import anytree
from anytree import Node, RenderTree

In [13]:
node_depth = np.zeros(shape=n_nodes, dtype=np.int64)
parents = np.zeros(shape=n_nodes, dtype=np.int64)
is_leaves = np.zeros(shape=n_nodes, dtype=bool)
stack = [(0, 0)]  # start with the root node id (0) and its depth (0)
parents[0]=-1
while len(stack) > 0:
    # `pop` ensures each node is only visited once
    node_id, depth = stack.pop()
    node_depth[node_id] = depth
    
    # If the left and right child of a node is not the same we have a split
    # node
    is_split_node = children_left[node_id] != children_right[node_id]
    # If a split node, append left and right children and depth to `stack`
    # so we can loop through them
    if is_split_node:
        stack.append((children_left[node_id], depth + 1))
        stack.append((children_right[node_id], depth + 1))
        parents[children_left[node_id]]=node_id
        parents[children_right[node_id]]=node_id
    else:
        is_leaves[node_id] = True
    
print(parents)

[-1  0  1  1  0  4  4]


In [14]:
from anytree.exporter import DotExporter
l=list()
del_l=list()
for i in range(n_nodes):
    if is_leaves[i]:
        if (i+1)<n_nodes and is_leaves[i+1]:
            left=children_left[parents[i]]
            right=children_right[parents[i]]
            gerror_before = samples[parents[i]]-np.max(value[parents[i]])+ 0.5* (n_nodes-2)
            gerror_after = (samples[left]-np.max(value[left])+samples[left]-np.max(value[left]))+0.5*n_nodes
            if gerror_before<gerror_after:
                del_l.append(left)
                del_l.append(right)
                is_leaves[parents[i]]=True

node_list=[n_nodes]

str = "node={node}\n{feature}<={threshold}\ngini={error}\nsamples={sample}\nvalue ={values}".format(
                node=0,
                feature=list2[feature[0]],
                threshold=threshold[0],
                error=impurity[0],
                sample=samples[0],
                values=value[0],)
root=Node(str)
node_list.insert(0,root)
node_list_num=1
for i in range (1,n_nodes):
    NUM=0
    if(i in del_l):
        continue
    for j in range (0,len(del_l)):
        if(parents[i]>del_l[j]):
            NUM+=1
    parents[i]-=NUM
    if is_leaves[i]:
        str = "node={node}\ngini={error}\nsamples={sample}\nvalue ={values}".format(
                node=node_list_num,
                error=impurity[i],
                sample=samples[i],
                values=value[i],)
    else:
        str = "node={node}\n{feature}<={threshold}\ngini={error}\nsamples={sample}\nvalue ={values}".format(
                    node=node_list_num,
                    feature=list2[feature[i]],
                    threshold=threshold[i],
                    error=impurity[i],
                    sample=samples[i],
                    values=value[i],)
    node_list.insert(node_list_num,Node(str, parent=node_list[parents[i]]))
    node_list_num+=1
DotExporter(root).to_picture("root.png")